# Precomupte forward model results

In [5]:
import pyvista as pv
pv.set_jupyter_backend('static')

In [6]:
from functools import lru_cache
import os

import numpy as np
import xarray as xr

import cedalion.datasets
import cedalion.geometry.segmentation
import cedalion.imagereco.forward_model as fw
import cedalion.io.forward_model
import cedalion.plots
xr.set_options(display_expand_data=False);

In [7]:
def compute_fluence_mcx(rec, head):
    geo3d_snapped_ijk = head.align_and_snap_to_scalp(rec.geo3d)

    fwm = cedalion.imagereco.forward_model.ForwardModel(
        head, geo3d_snapped_ijk, rec._measurement_lists["amp"]
    )

    fluence_all, fluence_at_optodes = fwm.compute_fluence_mcx(cuda=True)

    return fluence_all, fluence_at_optodes


def plot_fluence(rec, head, fluence_all, src, det, wl):
    geo3d_snapped_ijk = head.align_and_snap_to_scalp(rec.geo3d)

    f = fluence_all.loc[src, wl].values * fluence_all.loc[det, wl].values

    # clip fluence to smallest positive value and transform to log
    f[f <= 0] = f[f > 0].min()
    f = np.log10(f)

    vf = pv.wrap(f)

    plt = pv.Plotter()
    plt.add_volume(
        vf,
        log_scale=False,
        cmap="plasma_r",
        clim=(-10, 0),
    )
    cedalion.plots.plot_surface(plt, head.brain, color="w")
    cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk, show_labels=False)

    cog = head.brain.vertices.mean("label").values
    plt.camera.position = cog + [-300, 30, 150]
    plt.camera.focal_point = cog
    plt.camera.up = [0, 0, 1]

    plt.show()


@lru_cache
def get_colin27():
    SEG_DATADIR, mask_files, landmarks_file = (
        cedalion.datasets.get_colin27_segmentation()
    )

    head = fw.TwoSurfaceHeadModel.from_segmentation(
        segmentation_dir=SEG_DATADIR,
        mask_files=mask_files,
        landmarks_ras_file=landmarks_file,
    )

    return head


@lru_cache
def get_icbm152():
    SEG_DATADIR, mask_files, landmarks_file = (
        cedalion.datasets.get_icbm152_segmentation()
    )

    head = fw.TwoSurfaceHeadModel.from_surfaces(
        segmentation_dir=SEG_DATADIR,
        mask_files=mask_files,
        brain_surface_file=os.path.join(SEG_DATADIR, "mask_brain.obj"),
        landmarks_ras_file=landmarks_file,
        brain_face_count=None,
        scalp_face_count=None,
    )

    return head


# fluence_all, fluence_at_optodes = plot_fluence(rec, head, fluence_all, "S4", "D2", 760.)

In [ ]:
rec = cedalion.datasets.get_fingertappingDOT()
head = get_colin27()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)

cedalion.io.forward_model.save_fluence("fluence_fingertappingDOT_colin27.h5", fluence_all, fluence_at_optodes)

In [ ]:
rec = cedalion.datasets.get_fingertappingDOT()
head = get_icbm152()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)
cedalion.io.forward_model.save_fluence("fluence_fingertappingDOT_icbm152.h5", fluence_all, fluence_at_optodes)

In [ ]:
rec = cedalion.datasets.get_fingertapping()
head = get_colin27()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)
cedalion.io.forward_model.save_fluence("fluence_fingertapping_colin27.h5", fluence_all, fluence_at_optodes)

In [ ]:
rec = cedalion.datasets.get_fingertapping()
head = get_colin27()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)
cedalion.io.forward_model.save_fluence("fluence_fingertapping_icbm152.h5", fluence_all, fluence_at_optodes)

In [25]:
head = get_icbm152()
geo3d, landmarks, meas_list = cedalion.datasets.get_ninja_cap_probe() 
landmarks = landmarks.pint.quantify("mm")
display(landmarks)
geo3dc = xr.concat([landmarks, geo3d], dim="label")
geo3d_snapped_ijk = head.align_and_snap_to_scalp(geo3dc)
fwm = cedalion.imagereco.forward_model.ForwardModel(
    head, geo3d_snapped_ijk, meas_list
)

plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head.brain, color="w")
cedalion.plots.plot_surface(plt, head.scalp, opacity=.1)
cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk)
plt.show(interactive = True, jupyter_backend = 'html')
#fluence_all, fluence_at_optodes = fwm.compute_fluence_mcx(cuda=True)
#cedalion.io.forward_model.save_fluence("fluence_nn24_wholehead_icbm152.h5", fluence_all, fluence_at_optodes)

<xarray.DataArray (label: 329, digitized: 3)> Size: 8kB
[mm] 131.1 188.0 234.0 128.0 182.0 25.9 ... 100.0 127.0 33.9 112.0 121.0 32.9
Coordinates:
  * label    (label) <U6 8kB 'Nz' 'Iz' 'RPA' 'LPA' ... 'POO6' 'POO4' 'POO2'
    type     (label) object 3kB PointType.LANDMARK ... PointType.LANDMARK
Dimensions without coordinates: digitized

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [20]:
#adds unit mm to landmarks to not make it dimensionless
landmarksc = landmarks.pint.quantify("mm")

display(landmarksc)
display(geo3d)

# concatenates both xarrays landmarks and geo3d
geo3dc = xr.concat([landmarksc, geo3d], dim="label")
display(geo3dc)

<xarray.DataArray (label: 329, digitized: 3)> Size: 8kB
[mm] 131.1 188.0 234.0 128.0 182.0 25.9 ... 100.0 127.0 33.9 112.0 121.0 32.9
Coordinates:
  * label    (label) <U6 8kB 'Nz' 'Iz' 'RPA' 'LPA' ... 'POO6' 'POO4' 'POO2'
    type     (label) object 3kB PointType.LANDMARK ... PointType.LANDMARK
Dimensions without coordinates: digitized

<xarray.DataArray (label: 200, digitized: 3)> Size: 5kB
[mm] 150.6 148.9 233.2 109.8 148.8 233.1 ... 102.2 102.6 217.5 109.8 117.0 227.5
Coordinates:
  * label    (label) <U4 3kB 'S1' 'S2' 'S3' 'S4' ... 'D141' 'D142' 'D143' 'D144'
    type     (label) object 2kB PointType.SOURCE ... PointType.DETECTOR
Dimensions without coordinates: digitized

Magnitude,[[131.1 188.0 234.0] [128.0 182.0 25.9] [42.9 192.0 128.0] ... [100.298325 133.355614 227.553106] [102.227894 102.568911 217.497089] [109.789724 117.04117 227.506845]]
Units,millimeter
